#### 0501_Lab  
1. Cleanse categorical data in the Student table in BTech22_\<your name\>.sqlite.   

In [19]:
import pandas as pd
import os
import sqlite3 as lite
from glob import glob

dbpath = 'BTech22_reethurgt.sqlite'
con = lite.connect(dbpath)

In [20]:
df=pd.read_excel("/Users/reethu/coding/college/DBMS/Data/BTech22 Data.xlsx")

In [21]:
df.to_sql('Students', con, index=False, if_exists='append')

192

In [22]:
con.execute("SELECT * FROM sqlite_master").fetchall()

[('table',
  'Students',
  'Students',
  2,
  'CREATE TABLE "Students" (\n"USN" TEXT,\n  "Name" TEXT,\n  "Email" TEXT,\n  "Section" TEXT,\n  "Sex" TEXT,\n  "DoB" TEXT,\n  "Height_cm" TEXT,\n  "Weight_Kg" TEXT,\n  "Home_State" TEXT,\n  "Home_Town" TEXT,\n  "Mother_Tongue" TEXT,\n  "Elective" TEXT,\n  "Clubs" TEXT\n)')]

In [23]:
con.execute("CREATE TABLE Students_temp AS SELECT * FROM Students")
con.commit()

In [24]:
pd.read_sql("SELECT COUNT(*) FROM Students_temp", con)

,COUNT(*)
0,192


In [25]:
pd.read_sql("PRAGMA table_info(Students_temp)", con)

,cid,name,type,notnull,dflt_value,pk
0,0,USN,TEXT,0,None,0
1,1,Name,TEXT,0,None,0
2,2,Email,TEXT,0,None,0
3,3,Section,TEXT,0,None,0
4,4,Sex,TEXT,0,None,0
5,5,DoB,TEXT,0,None,0
6,6,Height_cm,TEXT,0,None,0
7,7,Weight_Kg,TEXT,0,None,0
8,8,Home_State,TEXT,0,None,0
9,9,Home_Town,TEXT,0,None,0


In [26]:
con.execute("SELECT DISTINCT Home_State FROM Students_temp").fetchall()

[('Karnataka ',),
 (None,),
 ('Karnataka',),
 ('Jammu',),
 ('Haryana',),
 ('karnataka ',),
 ('Delhi',),
 ('karnataka',),
 ('West Bengal',),
 ('Andhra Pradesh',),
 ('Telangana',),
 ('Punjab',),
 ('Goa',),
 ('Uttar Pradesh',),
 ('Rajasthan ',),
 ('Tamil Nadu',),
 ('Bihar',),
 ('Nil ',),
 ('Andhra Pradesh ',),
 ('Uttar Pradesh ',),
 ('KARNATAKA',),
 ('karntaka',),
 ('Gujarat',),
 ('Jharkhand',),
 ('-',),
 ('Rajasthan',),
 ('KARnataka',),
 ('Punjab ',),
 ('Karantaka',)]

In [27]:
query1 = """
    UPDATE Students_temp
    SET Home_State = TRIM(Home_State)
"""
query2 = """
    UPDATE Students_temp
    SET Home_State = 'Karnataka'
    WHERE Home_State IN
    ('karnataka','KARNATAKA','karntaka','KARnataka','Karantaka')
"""
query3 = """
    UPDATE Students_temp
    SET Home_State = 'unknown'
    WHERE Home_State IN ('unknown', '-', 'Nil', 'None')
"""
query4 = """
    UPDATE Students_temp
    SET Home_State = 'unknown'
    WHERE Home_State IS NULL;
"""

con.execute(query1)
con.execute(query2)
con.execute(query3)
con.execute(query4)
con.commit()


In [28]:
con.execute("SELECT DISTINCT Home_State FROM Students_temp").fetchall()

[('Karnataka',),
 ('unknown',),
 ('Jammu',),
 ('Haryana',),
 ('Delhi',),
 ('West Bengal',),
 ('Andhra Pradesh',),
 ('Telangana',),
 ('Punjab',),
 ('Goa',),
 ('Uttar Pradesh',),
 ('Rajasthan',),
 ('Tamil Nadu',),
 ('Bihar',),
 ('Gujarat',),
 ('Jharkhand',)]

In [29]:
con.execute("SELECT DISTINCT Mother_Tongue FROM Students_temp").fetchall()

[('Telugu',),
 ('English ',),
 ('Marathi',),
 ('Urdu',),
 ('Kannada',),
 ('Hindi',),
 (None,),
 ('kannada',),
 ('Kannada ',),
 ('Hinglish',),
 ('Marathi ',),
 ('Konkani',),
 ('Tamil',),
 ('Malayalam',),
 ('Hindi ',),
 ('kannada ',),
 ('tamil',),
 ('Tulu',),
 ('Karnataka',),
 ('MARATHI',),
 ('hindi',),
 ('telugu',),
 ('Gujarati',),
 ('Telugu ',),
 ('-',),
 ('KANNADA',),
 ('Punjabi',),
 ('Urdu ',),
 ('ಕನ್ನಡ',)]

In [33]:
query1 = """
    UPDATE Students_temp
    SET Mother_Tongue = TRIM(Mother_Tongue)
"""
query2 = """
    UPDATE Students_temp
    SET Mother_Tongue = 'Telugu'
    WHERE Mother_Tongue = 'telugu'
"""
query3 = """
    UPDATE Students_temp
    SET Mother_Tongue = 'Kannada'
    WHERE Mother_Tongue IN ('KANNADA','ಕನ್ನಡ','Karnataka','kannada')
"""
query4 = """
    UPDATE Students_temp
    SET Mother_Tongue = 'Marathi'
    WHERE Mother_Tongue = 'MARATHI'
"""
query5 = """
    UPDATE Students_temp
    SET Mother_Tongue = 'Hindi'
    WHERE Mother_Tongue = 'hindi'
"""
query6 = """
    UPDATE Students_temp
    SET Mother_Tongue = 'Tamil'
    WHERE Mother_Tongue = 'tamil'
"""
query7 = """
    UPDATE Students_temp
    SET Mother_Tongue = 'unknown'
    WHERE Mother_Tongue IN ('unknown', '-', 'Nil', 'None')
"""
query8 = """
    UPDATE Students_temp
    SET Mother_Tongue = 'unknown'
    WHERE Mother_Tongue IS NULL;
"""

con.execute(query1)
con.execute(query2)
con.execute(query3)
con.execute(query4)
con.execute(query5)
con.execute(query6)
con.execute(query7)
con.execute(query8)
con.commit()
con.execute("SELECT DISTINCT Mother_Tongue FROM Students_temp").fetchall()


[('Telugu',),
 ('English',),
 ('Marathi',),
 ('Urdu',),
 ('Kannada',),
 ('Hindi',),
 ('unknown',),
 ('Hinglish',),
 ('Konkani',),
 ('Tamil',),
 ('Malayalam',),
 ('Tulu',),
 ('Gujarati',),
 ('Punjabi',)]

2. Create look-up tables (aka master tables) for states, towns, and languages.    
Insert data.     
See [RTO Codes](https://www.v3cars.com/car-guide/state-code-regional-transport-office-rto-india) and 
Census Codes for [States and Cities](https://censusindia.gov.in/nada/index.php/catalog/42648) and 
[Language Codes](https://censusindia.gov.in/nada/index.php/catalog/42561)  
Use standard codes or abbreviations as PK.  



In [34]:
con.execute("CREATE TABLE Master_Lang(Language TEXT PK, Code TEXT)")
con.execute("CREATE TABLE Master_State(State TEXT PK, Code TEXT)")
con.execute("CREATE TABLE Master_City(City TEXT PK, State_code TEXT, District_code TEXT)")
con.execute("CREATE TABLE Master_Elective(ID TEXT PK, Elective TEXT)")
con.execute("CREATE TABLE Master_Club(ID TEXT PK, Club TEXT)")

In [35]:
rows1 = [('Karnataka','KA'),
 ('Jammu','JK'),
 ('Haryana','HR'),
 ('Delhi','DL'),
 ('West Bengal','WB'),
 ('Andhra Pradesh','AP'),
 ('Telangana','TS'),
 ('Punjab','PB'),
 ('Goa','GA'),
 ('Uttar Pradesh','UP'),
 ('Rajasthan','RJ'),
 ('Tamil Nadu','TN'),
 ('Bihar','BR'),
 ('Unknown','Unknown'),
 ('Gujarat','GJ'),
 ('Jharkhand','JH')]

con.executemany("INSERT INTO Master_State(State,Code) VALUES(?,?)",rows1)

In [36]:
rows2 = [('Mysuru',577, 5572),
 ('Bengaluru',572,000),
 ('Bidar',574,5558),
 ('Bathindi',0,0),
 ('Ballari',572,54432),
 ('Raichur',559,000),
 ('Hubballi',562,5475),
 ('Mandya',573,550),
 ('Chitradurga',566,000),
 ('Kalaburgi',677,2322),
 ('Maahishmati',720,13455),
 ('Hosapete',554,23111),
 ('New Delhi',1,000),
 ('Dharwad',563,65443),
 ('Kolkata',456,12364),
 ('Tirupati',245,13453),
 ('Chikkamagaluru',533,4546),
 ('Shivamogga',523,12253),
 ('Hyderabad',252,6553),
 ('Adoni',765,3312),
 ('Nagoor',332,75234),
 ('Amritsar',124,32345),
 ('Goa',597,6536),
 ('Bidadi',553,13443),
 ('Saharanpur',311,564),
 ('Abu Road',00,00),
 ('Chennai',653,13354),
 ('Haveri',524,43543),
 ('Yadgir',592,93489),
 ('Tumkur',642,45211),
 ('Dharbanga',674,7653),
 ('Davengere',906,46222),
 ('None',00,00),
 ('Lucknow',567,13534),
 ('Chikkaballapur',876,56533),
 ('Gangavathi',977,3132),
 ('Vijayapuraa',564,1329),
 ('Ahmedabad',1333,6756),
 ('Chittoor',233,5676),
 ('Mudigere',133,65655),
 ('Kolar',454,43544),
 ('Gajendragad',565,2323),
 ('Jamshedpur',565,7654),
 ('Belagavi',554,5643),
 ('Hassan',243,56765),
 ('Mangalore',231,56754),
 ('Magadi',654,24353),
 ('Ananthapur',657,2345),
 ('Vijayapura',765,52324),
 ('Jaipur',675,23475)]

con.executemany("INSERT INTO Master_City(city,State_Code,District_code) VALUES(?,?,?)",rows2)

In [37]:
rows3 = [('Telugu','te'),
 ('English','en'),
 ('Marathi','mr'),
 ('Urdu','ur'),
 ('Kannada','kn'),
 ('Hindi','hi'),
 ('Hinglish','he'),
 ('Konkani','kok'),
 ('Tamil','ta'),
 ('Malayalam','ml'),
 ('Tulu','tl'),
 ('Gujarati','gu'),
 ('Punjabi','pa')]

con.executemany("INSERT INTO Master_Lang(Language, Code) VALUES(?,?)",rows3)

In [38]:
query = """
  UPDATE Students_temp
     SET Home_State = Master_State.code
     FROM Master_State
        WHERE Master_State.State = Students_temp.Home_State
"""
con.execute(query)
con.commit()

In [39]:
sql = """
    ALTER TABLE Students_temp
    RENAME COLUMN Home_State TO state_id;
"""
con.execute(sql)

In [40]:
pd.read_sql("SELECT Name,Section,state_id FROM Students_temp", con)

,Name,Section,state_id
0,SHREE VYSHNAVI A,B,KA
1,AARON ARTHUR ELISHA,A,unknown
2,ABHISHEK,A,KA
3,ABRAR HUSSAIN,B,JK
4,ADITI A,B,KA
5,ADITYA GARG,C,HR
6,ADITYA ADUKURI,B,KA
7,ADITYA KUSHAL,A,KA
8,ADITYA P,A,KA
9,ADITYA RAJU A,A,KA


In [42]:
query = """
  UPDATE Students_temp
     SET Mother_Tongue = Master_Lang.code
     FROM Master_Lang
        WHERE Master_Lang.Language = Students_temp.Mother_Tongue
"""
con.execute(query)
con.commit()

In [43]:
sql = """
    ALTER TABLE Students_temp
    RENAME COLUMN Mother_Tongue TO Language_id;
"""
con.execute(sql)

In [44]:
pd.read_sql("SELECT Name,Section,Language_id FROM Students_temp", con)

,Name,Section,Language_id
0,SHREE VYSHNAVI A,B,te
1,AARON ARTHUR ELISHA,A,en
2,ABHISHEK,A,mr
3,ABRAR HUSSAIN,B,ur
4,ADITI A,B,kn
5,ADITYA GARG,C,hi
6,ADITYA ADUKURI,B,te
7,ADITYA KUSHAL,A,kn
8,ADITYA P,A,unknown
9,ADITYA RAJU A,A,te


3. Create tables for electives and clubs.  
    Their PK may be auto-incremented.  
    Insert data.  

In [45]:
query = """
    UPDATE Students_temp
    SET elective = TRIM(elective)
"""
con.execute(query)

In [46]:
con.execute("SELECT DISTINCT elective FROM Students_temp").fetchall()

[('Photography',),
 ('Community service',),
 ('Drumming',),
 ('Indian Knowledge System: Mathematica and Ayurveda',),
 ('Success, Happiness and Leadership',),
 ('Hindusthani Vocals',),
 ('Contemporary dance',),
 (None,),
 ('Contemporary ance',),
 ('Contemporary Dance',),
 ('contemporary dance',),
 ('Mixed martial arts',),
 ('Success,Happiness,Leadership and Lifeskills',),
 ('Community Service',),
 ('photography',),
 ('Success, Hapiness and Leadership',),
 ('Quantitative Reasoning',),
 ('Quantitative reasoning',),
 ('Indian Knowledge systems - Ayurveda and Mathematics',),
 ('Quantity Reasoning',),
 ('success, happiness and leadership and life skills',),
 ('Hindustani Vocals',),
 ('Success,Happiness,Leadership and  life skills',),
 ('Hindustani vocals',),
 ('Null',),
 ('success, happiness and leadership',),
 ('success,happiness,leadership and life skills',),
 ('community servive',),
 ('drumming',),
 ('Kuchipudi',),
 ('community service',),
 ('Success, happiness,leadership and life skills'

In [47]:
ele = [(1,'Photography '),
 (2,'Community service '),(3,'Drumming'),(4,'Indian Knowledge System: Mathematica and Ayurveda'),
 (5,'Success, Happiness and Leadership'),
 (6,'Hindusthani Vocals'),
 (7,'Contemporary dance '),(8,'Mixed martial arts'),(9,'Kuchipudi'),]

In [49]:
con.executemany("INSERT INTO Master_Elective(ID, Elective) VALUES(?,?)",ele)

In [54]:
pd.read_sql("SELECT * FROM Master_Elective",con)

,ID,Elective
0,1,Photography
1,2,Community service
2,3,Drumming
3,4,Indian Knowledge System: Mathematica and Ayurveda
4,5,"Success, Happiness and Leadership"
5,6,Hindusthani Vocals
6,7,Contemporary dance
7,8,Mixed martial arts
8,9,Kuchipudi


In [50]:
club = [(1,'cybersecurity'), (2,'Neural Nexus') ,(3,'Viksha'),(4,'callisto astronomy club'),(5,'ಶ್ರೀಗಂಧ'),(6,'Possessor Architecture'),(7,'carvaccess(rvce)'),(8,'E-cell'),(9,'expressions'), (10,'Raag'), (11,'generative AI'),(12,'Q-Pulse',),
 (13,'Rowlings Elf',),(14,'Socials'),(15,'AMEYA'),(16,'Debsoc'),(17,'Happy tails')]

In [52]:
con.executemany("INSERT INTO Master_Club(ID, Club) VALUES(?,?)",club)

In [53]:
pd.read_sql("SELECT * FROM Master_Club",con)

,ID,Club
0,1,cybersecurity
1,2,Neural Nexus
2,3,Viksha
3,4,callisto astronomy club
4,5,ಶ್ರೀಗಂಧ
5,6,Possessor Architecture
6,7,carvaccess(rvce)
7,8,E-cell
8,9,expressions
9,10,Raag


In [55]:
con.commit()
con.close()